# Install library

In [ ]:
!pip install -q datasets==2.16.0
!pip install -q bitsandbytes
!pip install -q tiktoken
!pip install -q peft
!pip install -q trl
!pip install -q transformers
!pip install -q openpyxl
!pip install -q pandas
!pip install -q scikit-learn
!pip install -q flash-attn
#pip install -q transformers==4.38.1

# Import library

In [ ]:
import json
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from accelerate import PartialState
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from peft import prepare_model_for_kbit_training
from transformers import TrainingArguments

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

# Hyperparameters

In [ ]:
modelpath = "meta-llama/Meta-Llama-3-8B-Instruct"
lr=2e-4      # learning rate
bs=16            # batch size
bs_eval=16      # batch size for evals
ga_steps=1     # gradient acc. steps
epochs=4
max_length=128      # max. sample length with 24GB VRAM
output_dir="out"

# Remove old model
Because of limited storage, we can't save all models, we need to delete all models in cache by the following code:
- rm -r out: delete out folder (because I save finetuned model in out folder), you can change folder name like (rm -r output_folder). If you doesn't have out folder, this commend do not thing.
- all pretrained huggingface models will auto save in transformers.TRANSFORMERS_CACHE. I use glob.glob to read all folders and use shutil.rmtree to delete them.

In [ ]:
!rm -r out

In [ ]:
# from transformers import TRANSFORMERS_CACHE
# import glob
# print(TRANSFORMERS_CACHE)
# folders = glob.glob(f"{TRANSFORMERS_CACHE}/*")
# print(folders)

# import shutil
# #shutil.rmtree(TRANSFORMERS_CACHE)
# for folder in folders:
#     if 'Phi-3' not in folder and '.txt' not in folder:
#         shutil.rmtree(folder)

# Create Dataset
Download dataset from [kaggle synthetic-vietnamese-students-feedback-corpus](https://www.kaggle.com/datasets/toreleon/synthetic-vietnamese-students-feedback-corpus/data)

We need convert DataFrame to json line (jsonl).

In [ ]:
df_train = pd.read_csv("synthetic_train.csv")
df_test = pd.read_csv("synthetic_val.csv")

In [ ]:
df_train.head()

,sentence,sentiment,topic
0,Đội ngũ bảo trì quá thưa thớt dẫn đến không đả...,negative,facility
1,The university's musical and artistic faciliti...,neutral,facility
2,Phương pháp giảng dạy phù hợp với các đối tượn...,neutral,curriculum
3,Chương trình học giúp tôi trở thành một chuyên...,positive,curriculum
4,Tôi nghĩ rằng chương trình đào tạo có thể có t...,neutral,curriculum


In [ ]:
df_test.head()

,sentence,sentiment,topic
0,Chất lượng vật chất kém.,negative,facility
1,"Phần mềm học tập quá khó sử dụng, khiến sinh v...",negative,facility
2,Trường tôi thiếu những tiện ích cơ bản như máy...,negative,facility
3,Cần tạo thêm các hoạt động gắn kết giữa sinh v...,neutral,curriculum
4,Họ rất khoan dung và lượng giác trong quan điể...,neutral,others


In [ ]:
df_train.sentiment.value_counts()

sentiment
neutral     2724
negative    2711
positive    2709
Name: count, dtype: int64

In [ ]:
df_test.sentiment.value_counts()

sentiment
negative    686
positive    680
neutral     670
Name: count, dtype: int64

In [ ]:
df_train['len'] = df_train.sentence.apply(lambda x: len(str(x).split()))
df_test['len'] = df_test.sentence.apply(lambda x: len(str(x).split()))

In [ ]:
df_train['len'].describe()

count    8144.000000
mean       15.549730
std         5.018764
min         3.000000
25%        12.000000
50%        15.000000
75%        18.000000
max        43.000000
Name: len, dtype: float64

In [ ]:
df_test['len'].describe()

count    2036.000000
mean       15.694990
std         5.185957
min         2.000000
25%        12.000000
50%        15.000000
75%        19.000000
max        48.000000
Name: len, dtype: float64

In [ ]:
with open('train.jsonl', 'w') as outfile:
    for i, x in df_train.iterrows():
        comment = x['sentence']
        label = x['sentiment']
        #label = 'yes' if label == 'relevance' else 'no'
        data = {
            "input": f'''The sentiment of this comment "{comment}" is''',
            "output": f"{label}"
        }
        json.dump(data, outfile)
        outfile.write('\n')

In [ ]:
with open('test.jsonl', 'w') as outfile:
    for i, x in df_test.iterrows():
        comment = x['sentence']
        label = x['sentiment']
        #label = 'yes' if label == 'relevance' else 'no'
        data = {
            "input": f'''The sentiment of this comment "{comment}" is''',
            "output": f"{label}"
        }
        json.dump(data, outfile)
        outfile.write('\n')

In [ ]:
data_files = {
    "train": "train.jsonl",
    "validation": "test.jsonl",
}

dataset = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 8144
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 2036
    })
})

# Create prompt format
Load tokenizer by AutoTokenizer.from_pretrained:
- We need create and copy YOUR TOKEN from [huggingface](https://huggingface.co/settings/tokens)
- With llama 3, we need add padding token (many others llm doesn't need this step): tokenizer.pad_token = tokenizer.eos_token
- We need use padding_side = 'right' because training library need padding_side = 'right' when training. You can use left padding but you need to make sure the library is not corrupted and check whether performance is affected by left padding!
- If you have 1 prompt like "test thử mô hình" and want to tokenize it, just you tokenizer(prompt, return_tensors="pt"). You can see output of tokenizer in cell below (output includes input_ids (list index of each token in prompt) and attention mask)
- We can use tokenizer.batch_decode to see how tokenizer restore string from token tensor. You can see that it automatically adds the start token "<bos>" at the beginning of the string.
- To train llm, we only need to pass 1 sentence to llm (including input and desired output) without specifying which is the input and which is the output.
- I wrote the function formatting_prompts_func to convert input and output to prompt and tested this function, you can see below.
- When predicting, remove the output part to let the model predict itself. See the predict section below later.
- we only need to predict some next tokens like A. positive, and B. neutral. We don't care what the model says after sentiment. Then we do not need to add <eos token>. If you fine-tune the model with other tasks, maybe you need to add <eos token> at the end of the prompt:
  - use tokenizer.eos_token, tokenizer.eos_token_id to see eos_token of your model and correspond id
  - for ex: eos_token is "<|im_end|>". You need edit prompt like:
    - '''...The correct answer is {output_}.''' --> '''...The correct answer is {output_}. <|im_end|>'''
  - for ex: eos_token is "end_token__". You need edit prompt like:
    - '''...The correct answer is {output_}.''' --> '''...The correct answer is {output_}. end_token__'''

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    modelpath,
    padding_side="right",
    # add_eos_token=True,
    # add_bos_token=True,
    trust_remote_code=True,
    token = 'YOUR TOKEN HERE'
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
prompt = "test thử mô hình"
tokens = tokenizer(prompt, return_tensors="pt")
print(tokens)
tokenizer.batch_decode(tokenizer.encode(prompt))

{'input_ids': tensor([[128000,   1985, 110832, 105158, 101157]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


['<|begin_of_text|>', 'test', ' thử', ' mô', ' hình']

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['input'])):
        input_ = example['input'][i]
        output_ = example['output'][i]
        output_ = 'A. Positive' if output_ == 'positive' else 'B. Neutral' if output_ == 'neutral' else 'C. Negative'
        #text = f"### Question: {input__}\n ### Answer: {example['output'][i]}"
        text = f'''{input_}
A. Positive
B. Neutral
C. Negative

The correct answer is {output_}.'''

        output_texts.append(text)
    return output_texts

In [ ]:
dataset["validation"]['input'][0], dataset["validation"]['output'][0]

('The sentiment of this comment "Chất lượng vật chất kém." is', 'negative')

In [ ]:
formatting_prompts_func({'input': [dataset["validation"]['input'][0]],
                         'output': [dataset["validation"]['output'][0]]})

['The sentiment of this comment "Chất lượng vật chất kém." is\nA. Positive\nB. Neutral\nC. Negative\n\nThe correct answer is C. Negative.']

In [ ]:
print(formatting_prompts_func({'input': [dataset["validation"]['input'][0]],
                         'output': [dataset["validation"]['output'][0]]})[0])

The sentiment of this comment "Chất lượng vật chất kém." is
A. Positive
B. Neutral
C. Negative

The correct answer is C. Negative.


# Create model

## Load model
Because we use 4 bit for training, we need use BitsAndBytesConfig:
- load_in_4bit = True: we will load model with 4 bit format.
- bnb_4bit_use_double_quant = True: use double quant (you can search how double quant work)
- bnb_4bit_quant_type = 'nf4': nf4 is the normalized float 4 bit data type. It quantizes floats into 4 bits (you can search about this)
- bnb_4bit_compute_dtype: dtype before quantize (we use bfloat16, if your gpu don't support bfloat16, set it to float32).

We use AutoModelForCausalLM.from_pretrained to load model:
- device_map = 'auto': auto active gpu.
- torch_dtype: use bfloat16, if your gpu don't support bfloat16, set it to float32
- quantization_config: config of quantization (bnb_config above)
- attn_implementation: you can use 'flash_attention_2', if you meet bug, maybe your gpu doesn't support it, you need delete this line.
- token: you huggingface token like tokenizer above.

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    modelpath,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    #torch_dtype=torch.float32,
    quantization_config=bnb_config,
    #attn_implementation="flash_attention_2",
    trust_remote_code=True,
    token = 'YOUR TOKEN HERE'
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#model

## Eval model before training
We use create_prompt function to generate prompt (without output), our model will predict output. We will eval model before finetune. You can see some predict below.

You can see that pretrained model can't clarify neutral class!

In [ ]:
def create_prompt(input_, output_):
    output_ = 'A. Positive' if output_ == 'positive' else 'B. Neutral' if output_ == 'neutral' else 'C. Negative'
        #text = f"### Question: {input__}\n ### Answer: {example['output'][i]}"
    text = f'''{input_}
A. Positive
B. Neutral
C. Negative

The correct answer is'''

    return text

sentence = 'món ăn rất ngon'
input_ = f'''The sentiment of this comment: "{sentence}" is'''
prompt = create_prompt(input_, "")
print(prompt)

inputs = torch.tensor([tokenizer.encode(prompt)])

tokens = model.generate(
    inputs.to(model.device),
    max_new_tokens=50,
    temperature=0.1,
    do_sample=False
)
print(tokenizer.decode(tokens[0], skip_special_tokens=False))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpec

The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is
<|begin_of_text|>The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive.

The comment "món ăn rất ngon" means "the food is very delicious" in Vietnamese. The word "rất" means "very" and "ngon" means "delicious". The tone is positive, indicating


In [ ]:
from datetime import datetime

start = datetime.now()

prediction = []
response = []
accuracy = []
labels = []

for i, x in df_test.iterrows():
    sentence = x['sentence']
    label = x['sentiment']
    input_ = f'''The sentiment of this comment: "{sentence}" is'''
    prompt = create_prompt(input_, label)

    inputs = tokenizer.encode(
        prompt,
        # add_generation_prompt=True,
        return_tensors='pt'
    )

    tokens = model.generate(
        inputs.to(model.device),
        max_new_tokens=5,
        temperature=0.1,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    #break

    answer = tokenizer.decode(tokens[0], skip_special_tokens=False).split("The correct answer is ")[-1]
    answer = 'positive' if 'positive' in answer.lower() else 'negative' if 'negative' in answer.lower() else 'neutral'
    prediction.append(answer.lower())
    response.append(tokenizer.decode(tokens[0], skip_special_tokens=False))

    accuracy.append(prediction[-1] == label)
    labels.append(label)

    if i % 100 == 0:
        print(i, np.array(accuracy).sum()/len(prediction)*100, datetime.now() - start, (datetime.now() - start)/len(prediction))

0 100.0 0:00:00.340781 0:00:00.340785
100 67.32673267326733 0:00:32.942426 0:00:00.326163
200 69.15422885572139 0:01:05.599160 0:00:00.326364
300 67.109634551495 0:01:37.860561 0:00:00.325118
400 67.83042394014963 0:02:10.558754 0:00:00.325583
500 68.8622754491018 0:02:43.090409 0:00:00.325530
600 67.38768718801997 0:03:15.780670 0:00:00.325758
700 66.47646219686163 0:03:48.681153 0:00:00.326221
800 67.16604244694132 0:04:20.654184 0:00:00.325411
900 66.2597114317425 0:04:52.973676 0:00:00.325165
1000 66.63336663336663 0:05:25.464170 0:00:00.325139
1100 66.30336058128974 0:05:57.976844 0:00:00.325138
1200 65.94504579517069 0:06:30.584747 0:00:00.325216
1300 66.25672559569563 0:07:02.770622 0:00:00.324958
1400 66.66666666666666 0:07:34.674831 0:00:00.324536
1500 66.75549633577616 0:08:07.283267 0:00:00.324639
1600 67.02061211742661 0:08:39.662462 0:00:00.324586
1700 67.13697824808936 0:09:12.284005 0:00:00.324682
1800 67.46252082176568 0:09:43.785290 0:00:00.324145
1900 67.4381904260915

In [ ]:
from sklearn.metrics import classification_report, f1_score
import sklearn

print(sklearn.metrics.classification_report(labels, prediction, digits=4))

              precision    recall  f1-score   support

    negative     0.9494    0.9854    0.9671       686
     neutral     0.8000    0.0299    0.0576       670
    positive     0.5181    0.9897    0.6801       680

    accuracy                         0.6724      2036
   macro avg     0.7558    0.6683    0.5683      2036
weighted avg     0.7562    0.6724    0.5719      2036



In [ ]:
for x in response[-10:]:
    print(x)
    print("\n")

<|begin_of_text|>The sentiment of this comment: "The facilities of the university are versatile and helpful." is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive. The


<|begin_of_text|>The sentiment of this comment: "Mấy bạn đó hay đòi hỏi nhưng không bao giờ giúp đỡ người khác." is
A. Positive
B. Neutral
C. Negative

The correct answer is C. Negative.

The


<|begin_of_text|>The sentiment of this comment: "Cậu ấy rất có kỹ năng về sáng tạo và nghệ thuật." is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive.

The


<|begin_of_text|>The sentiment of this comment: "Giảng viên này không nhàm chán." is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive.

The


<|begin_of_text|>The sentiment of this comment: "Anh ta là một người rất tỉ mỉ và cẩn thận." is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive.

The


<|begin_of_text|>The sentiment of this comment: "Giáo viên đưa ra các phương tiện hỗ trợ giảng dạy 

## Create peft
- To training model with 4 bit, we need you prepare_model_for_kbit_training.
- For lora training, we need craete LoraConfig:
  - lora_alpha, r, dropout is basic hyperarameter.
  - almost LLM use bias = 'none' when finetune then we set is to None
  - target_modules are list of layer we want to finetune, I set it to 'all-linear', then all linear layers will be finetuned. If you just finetune some layers like q_proj, k_proj, you can pass list ['q_proj', 'k_proj'].
  - modules_to_save is other layers we want to finetune (but don't use lora), in my experience, finetune all embedding layers will make model work betters than I set modules_to_save as list of all embedding layers.

In [ ]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [ ]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = 'all-linear',
#     target_modules=["q_proj",
#         "k_proj",
#         "v_proj",
#         "o_proj",
#         "gate_proj",
#         "up_proj",
#         "down_proj",
#         "lm_head",],
    modules_to_save=["embed_tokens", "rotary_emb"]
                     #"input_layernorm", "post_attention_layernorm", "norm"]
)
model = get_peft_model(model, peft_config)

model.print_trainable_parameters() #

trainable params: 546,308,096 || all params: 8,576,569,344 || trainable%: 6.3698


In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(128256, 4096)
          (modules_to_save): ModuleDict(
            (default): Embedding(128256, 4096)
          )
        )
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): Par

## Create TrainingArguments
If your model doesn't support bfloat16, set bf16 = False and turn on fb16 = True.

In [ ]:
print(len(df_train)//bs//ga_steps*epochs//4)

509


In [ ]:
save_step = len(df_train)//bs//ga_steps*epochs//4
print(save_step)

509


In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs_eval,
    evaluation_strategy="steps",
    eval_steps=save_step,
    gradient_accumulation_steps=ga_steps,
    optim="paged_adamw_32bit",
    save_steps=save_step,
    save_strategy="steps",
    logging_steps=save_step,
    learning_rate=lr,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
    save_total_limit=1,
    #load_best_model_at_end=True
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Create trainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    max_seq_length= 128,
    #dataset_text_field=["input", "output"],
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
    formatting_func = formatting_prompts_func,
    #data_collator=collator
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFT

Map:   0%|          | 0/8144 [00:00<?, ? examples/s]

Map:   0%|          | 0/2036 [00:00<?, ? examples/s]

# Train

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
509,0.787100,0.762071
1018,0.677300,0.756534
1527,0.598600,0.779737
2036,0.526300,0.808275


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-667f7e11-72ce7f617234ad8961383e16;838352f9-cfee-43db-bec4-19ca8fcd3368)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B-Instruct.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B-Instruct - will ass

TrainOutput(global_step=2036, training_loss=0.6473308780572737, metrics={'train_runtime': 2814.4954, 'train_samples_per_second': 11.574, 'train_steps_per_second': 0.723, 'total_flos': 6.976975385906381e+16, 'train_loss': 0.6473308780572737, 'epoch': 4.0})

# Eval
- If you save the model to the output_dir folder, the training code above will automatically save the model to output_dir/checkpoint-{save_step} (in my case: out/checkpoint-2036). We will load the model from this folder:
  - I calculate _id = save_step * num_epoch = 509 * 4 = 2036
  - My model will saved at "{output_dir}/checkpoint-{_id}"
- I use del model, gc.collect() and torch.cuda.empty_cache() to release trained model (save gpu memory).
- We use PeftConfig.from_pretrained to lead peft config (this is the same as peft config at training)
- we load pretrained model like before.
- We load the tokenizer in this folder by passing the saved folder to AutoTokenizer.from_pretrained()
- We use PeftModel.from_pretrained(model, peft_model_id) to merge pre-trained model with finetuned lora model (note that when we use peft lora, training code only saves lora model, this saves our storage and saving time)

In [ ]:
save_step = len(df_train)//bs//ga_steps*epochs//4
print(save_step)

509


In [ ]:
import gc

del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

_id = save_step*4

peft_model_id = f"{output_dir}/checkpoint-{_id}"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    modelpath,
    device_map="auto",
    #torch_dtype=torch.float16,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    #attn_implementation="flash_attention_2",
    trust_remote_code=True,
    token = 'YOUR TOKEN HERE'
)

tokenizer = AutoTokenizer.from_pretrained(f"{output_dir}/checkpoint-{_id}",
                                          trust_remote_code=True,
                                          padding_side='left',
                                          token='YOUR TOKEN HERE')

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def create_prompt(input_, output_):
    output_ = 'A. Positive' if output_ == 'positive' else 'B. Neutral' if output_ == 'neutral' else 'C. Negative'
        #text = f"### Question: {input__}\n ### Answer: {example['output'][i]}"
    text = f'''{input_}
A. Positive
B. Neutral
C. Negative

The correct answer is'''

    return text

In [ ]:
sentence = 'món ăn rất ngon'
input_ = f'''The sentiment of this comment: "{sentence}" is'''
prompt = create_prompt(input_, "")
print(prompt)

The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is


In [ ]:
inputs = torch.tensor([tokenizer.encode(prompt)])
inputs

tensor([[128000,    791,  27065,    315,    420,   4068,     25,    330,     76,
           3244, 102607, 101750, 111495,      1,    374,    198,     32,     13,
          45003,    198,     33,     13,  59794,    198,     34,     13,  51957,
            271,    791,   4495,   4320,    374]])

In [ ]:
tokens = model.generate(
    inputs.to(model.device),
    max_new_tokens=50,
    temperature=0.1,
    do_sample=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(tokenizer.decode(tokens[0], skip_special_tokens=False))

<|begin_of_text|>The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive. Positive. Positive. Positive. Positive. Positive. Positive. Positive. Positive

TheThe
000
000. Positive. Positive Positive Positive

The correct answer is A. Positive. Positive. Positive. Positive. Positive. Positive


In [ ]:
tokens = model.generate(
    inputs.to(model.device),
    max_new_tokens=5,
    temperature=0.1,
    do_sample=False
)
print(tokenizer.decode(tokens[0], skip_special_tokens=False))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive. Positive


In [ ]:
from datetime import datetime

start = datetime.now()

prediction = []
response = []
accuracy = []
labels = []

for i, x in df_test.iterrows():
    sentence = x['sentence']
    label = x['sentiment']
    input_ = f'''The sentiment of this comment: "{sentence}" is'''
    prompt = create_prompt(input_, label)

    inputs = tokenizer.encode(
        prompt,
        # add_generation_prompt=True,
        return_tensors='pt'
    )

    tokens = model.generate(
        inputs.to(model.device),
        max_new_tokens=5,
        temperature=0.1,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    #break

    answer = tokenizer.decode(tokens[0], skip_special_tokens=False).split("The correct answer is ")[-1]
    answer = 'positive' if 'positive' in answer.lower() else 'negative' if 'negative' in answer.lower() else 'neutral'
    prediction.append(answer.lower())
    response.append(tokenizer.decode(tokens[0], skip_special_tokens=False))

    accuracy.append(prediction[-1] == label)
    labels.append(label)

    if i % 100 == 0:
        print(i, np.array(accuracy).sum()/len(prediction)*100, datetime.now() - start, (datetime.now() - start)/len(prediction))

0 100.0 0:00:00.458253 0:00:00.458259
100 87.12871287128714 0:00:44.732209 0:00:00.442893
200 87.56218905472637 0:01:29.650996 0:00:00.446025
300 86.37873754152824 0:02:13.887943 0:00:00.444810
400 85.785536159601 0:02:58.973963 0:00:00.446319
500 86.22754491017965 0:03:44.665193 0:00:00.448434
600 86.02329450915141 0:04:29.819706 0:00:00.448951
700 86.01997146932952 0:05:14.610638 0:00:00.448803
800 86.51685393258427 0:06:00.340824 0:00:00.449864
900 86.68146503884573 0:06:45.393008 0:00:00.449937
1000 86.81318681318682 0:07:29.057097 0:00:00.448608
1100 86.73932788374205 0:08:13.191496 0:00:00.447949
1200 85.67860116569526 0:08:57.202116 0:00:00.447296
1300 85.85703305149885 0:09:42.149202 0:00:00.447463
1400 86.36688079942898 0:10:26.320539 0:00:00.447052
1500 86.20919387075283 0:11:10.696658 0:00:00.446833
1600 85.94628357276703 0:11:55.076724 0:00:00.446644
1700 86.06701940035273 0:12:40.056063 0:00:00.446829
1800 86.17434758467517 0:13:24.715126 0:00:00.446816
1900 86.00736454497

In [ ]:
from sklearn.metrics import classification_report, f1_score
import sklearn

print(sklearn.metrics.classification_report(labels, prediction, digits=4))

              precision    recall  f1-score   support

    negative     0.9644    0.9883    0.9762       686
     neutral     0.8765    0.6567    0.7509       670
    positive     0.7473    0.9132    0.8220       680

    accuracy                         0.8541      2036
   macro avg     0.8627    0.8528    0.8497      2036
weighted avg     0.8630    0.8541    0.8505      2036



# Check results
- I print all sentences have wrong prediction and see that almost cases is bug.
- Conclusion, dataset is not clean than finetune LLM can't better than finetune roberta (~89..90%), because roberta will overfit even in test dataset.
- In additionally, when I print example: "món ăn này rất ngon", we can see that model before finetune work better. Model after finetune only think about school (because finetune dataset is about school) and it don't know about food review. Than I think we only need finetune for special cases and finetune dataset need be clean and large enough.

In [ ]:
df_test['predict'] = prediction

In [ ]:
for i, row in df_test.iterrows():
    if row['predict'] != row['sentiment']:
        print(row['sentence'], row['predict'], row['sentiment'])

Giảng viên luôn đồng hành với sinh viên trên quá trình học tập. positive neutral
Tôi có thể ứng dụng các kiến thức có được từ chương trình đào tạo vào công việc của mình. neutral positive
Giảng viên rất tài năng và có nhiều kinh nghiệm trong công tác giảng dạy. positive neutral
Được học tập với các giáo viên giàu kinh nghiệm và thực tiễn. positive neutral
Hệ thống hỗ trợ học tập trực tuyến rất tốt. positive neutral
Chị ấy rất giỏi quản lý thời gian và luôn hoàn thành công việc đúng tiến độ. positive neutral
Anh ấy có một tầm nhìn đỉnh cao và khả năng giải quyết vấn đề tốt. positive neutral
Các cửa hàng tiện lợi gần đó thường xuyên được mở cửa cho sinh viên. negative neutral
Trường đại học này đóng góp một phần quan trọng trong việc phát triển kinh tế và xã hội địa phương. positive neutral
Có nhiều hoạt động ngoại khóa và phong phú cho sinh viên. neutral positive
Khu vực đặt máy bán thức uống rất tiện lợi cho sinh viên. neutral positive
Sân cỏ của trường được tu bổ và định hướng riêng c